# Предобученные модели

На Huggingface доступно множество предобученных языковых моделей. 

https://huggingface.co/models

Следующие модели содержат русский язык.

## Сбербанк ruGPT

ai-forever/rugpt3small_based_on_gpt2 __125m__ \
ai-forever/rugpt3medium_based_on_gpt2 __356m__ \
ai-forever/rugpt3large_based_on_gpt2 __760m__ \
ai-forever/ruGPT-3.5-13B

## Сбербанк mGPT

ai-forever/mGPT __1.4b__

## Huggingface BLOOM
bigscience/bloom-560m\
bigscience/bloom-1b1\
bigscience/bloom-1b7\
bigscience/bloom-3b\
bigscience/bloom __366B__

## Facebook OPT
facebook/opt-125m\
facebook/opt-350m\
facebook/opt-1.3b\
facebook/opt-2.7b\
facebook/opt-6.7b\
facebook/opt-13b\
facebook/opt-30b\
facebook/opt-66b

## Meta LLaMA
meta-llama/Llama-2-7b-hf\
meta-llama/Llama-2-13b-hf\
meta-llama/Llama-2-70b-hf

## ... и другие


# Загрузка предобученной модели с HuggingFace

Выбор размера модели:

Допустим модель имеет 1b (миллиард) переменных

На диске она будет занимать от 2 до 4 Гб

В памяти GPU она будет занимать:

* 4Гб с опцией **torch_dtype=torch.float32**
* 2Гб с опцией **torch_dtype=torch.float16**
* 1Гб с опцией **load_in_8bit=True**
* 0.5Гб с опцией **load_in_4bit=True**

In [1]:
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig
import torch

# model_name = 'ai-forever/rugpt3small_based_on_gpt2'
# model_name = 'ai-forever/rugpt3large_based_on_gpt2'
model_name = 'ai-forever/ruGPT-3.5-13B'
# model_name = 'ai-forever/mGPT'
# model_name = 'facebook/opt-30b'
# model_name = 'bigscience/bloom-3b'
# model_name = 'meta-llama/Llama-2-7b-hf'

# model = AutoModelForCausalLM.from_pretrained(model_name, torch_dtype=torch.float16).to('cuda')
model = AutoModelForCausalLM.from_pretrained(model_name, load_in_4bit=True, bnb_4bit_compute_dtype=torch.float16)
tokenizer = AutoTokenizer.from_pretrained(model_name)

model_size = sum(t.numel() for t in model.parameters())
print(f"model_name: {model_name}")
print(f'model_size {model_size/1000**2:.1f}M')
print(tokenizer.special_tokens_map)

Loading checkpoint shards:   0%|          | 0/6 [00:00<?, ?it/s]

model_name: ai-forever/ruGPT-3.5-13B
model_size 6562.0M
{'bos_token': '<s>', 'eos_token': '</s>', 'unk_token': '<|endoftext|>', 'pad_token': '<pad>', 'mask_token': '<mask>'}


# Генерация с префиксом

In [2]:
prefix = 'О сколько нам открытий чудных'
# prefix = 'Вопрос: Сколько будет 2+2?\nОтвет:'
# prefix = 'По-русски: кот, по-английски:'
# prefix = 'Определение: "Нейронная сеть" - это'

inputs = tokenizer(prefix, return_tensors='pt').to('cuda')

outputs = model.generate(
    **inputs, 
    do_sample=True,
    # temperature=1.3,
    # num_beams=5,
    max_new_tokens=100,
    pad_token_id=tokenizer.eos_token_id,
    eos_token_id=tokenizer.unk_token_id,
    )

print(tokenizer.decode(outputs[0]))

О сколько нам открытий чудных... готовит просвещенья разум......не так ли?
ну если еще и про "чудесный мир", окружающий нас с вами (если не брать в рассмотрение "черную дыру" - мир "вещества", и "черную дыру" - мир "антивещества" - то есть мир, о наличие которого в мире "вещества" мы не можем знать...) - то да, пожалуй...но это будет уже философия, а здесь, как я понимаю, - все же психология


# Параметры модели

In [3]:
model

GPT2LMHeadModel(
  (transformer): GPT2Model(
    (wte): Embedding(50272, 5120)
    (wpe): Embedding(2048, 5120)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0-39): 40 x GPT2Block(
        (ln_1): LayerNorm((5120,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2Attention(
          (c_attn): Linear4bit(in_features=5120, out_features=15360, bias=True)
          (c_proj): Linear4bit(in_features=5120, out_features=5120, bias=True)
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((5120,), eps=1e-05, elementwise_affine=True)
        (mlp): GPT2MLP(
          (c_fc): Linear4bit(in_features=5120, out_features=20480, bias=True)
          (c_proj): Linear4bit(in_features=20480, out_features=5120, bias=True)
          (act): NewGELUActivation()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
    )
    (ln_f): LayerNorm((5120,), eps=1e-05, elem

In [4]:
model.config

GPT2Config {
  "_name_or_path": "ai-forever/ruGPT-3.5-13B",
  "activation_function": "gelu_new",
  "architectures": [
    "GPT2LMHeadModel"
  ],
  "attn_pdrop": 0.1,
  "bos_token_id": 50256,
  "embd_pdrop": 0.1,
  "eos_token_id": 50256,
  "initializer_range": 0.02,
  "layer_norm_epsilon": 1e-05,
  "model_type": "gpt2",
  "n_ctx": 2048,
  "n_embd": 5120,
  "n_head": 40,
  "n_inner": null,
  "n_layer": 40,
  "n_positions": 2048,
  "quantization_config": {
    "bnb_4bit_compute_dtype": "float16",
    "bnb_4bit_quant_type": "fp4",
    "bnb_4bit_use_double_quant": false,
    "llm_int8_enable_fp32_cpu_offload": false,
    "llm_int8_has_fp16_weight": false,
    "llm_int8_skip_modules": null,
    "llm_int8_threshold": 6.0,
    "load_in_4bit": true,
    "load_in_8bit": false,
    "quant_method": "bitsandbytes"
  },
  "reorder_and_upcast_attn": false,
  "resid_pdrop": 0.1,
  "scale_attn_by_inverse_layer_idx": false,
  "scale_attn_weights": true,
  "summary_activation": null,
  "summary_first_dro